<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
<a href="http://mng.bz/orYv">『スクラッチから大規模言語モデルを構築する』</a>（<a href="https://sebastianraschka.com">Sebastian Raschka</a>著）の補助コード<br>
<br>コードリポジトリ: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 第7章 演習解答

## 演習 7.1：プロンプトスタイルの変更

次のデータエントリがあるとします：

```json
{
  "instruction": "Identify the correct spelling of the following word.",
  "input": "Ocassion",
  "output": "The correct spelling is 'Occasion.'"
}
```

メイン章では、これをAlpacaスタイルのプロンプトテンプレートに従ってフォーマットしました：

```
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Identify the correct spelling of the following word.

### Input:
Occassion

### Response:
The correct spelling is 'Occasion.'
```

この演習では、代わりにPhi-3プロンプトテンプレートを使用し、データエントリを次のようにフォーマットします：

```
<user>
Identify the correct spelling of the following word: 'Occasion'

<assistant>
The correct spelling is 'Occasion'.
```

このプロンプトテンプレートは大幅に短く、入力プロンプトが短いため、LLMのファインチューニングとテキスト生成の実行時間とハードウェア要件を削減できることに注意してください。
この変更を行うため、`format_input`関数を次のように更新します：

In [1]:
def format_input(entry):
    instruction_text = (
        f"<|user|>\n{entry['instruction']}"
    )

    input_text = f"\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

2つの入力サンプル（`'input'`フィールドにコンテンツがあるものとないもの）に適用して、意図した通りに動作するか確認しましょう：

In [2]:
sample_data = [
    {'instruction': 'Identify the correct spelling of the following word.', 'input': 'Ocassion', 'output': "The correct spelling is 'Occasion.'"}, 
    {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."}
]

print(format_input(sample_data[0]))
print()
print(format_input(sample_data[1]))

<|user|>
Identify the correct spelling of the following word.
Ocassion

<|user|>
What is an antonym of 'complicated'?


次に、レスポンスに<|assistant|>プロンプトテンプレートを使用するように`InstructionDataset`クラスも更新します：

```python
import tiktoken
from torch.utils.data import Dataset

class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        # テキストを事前にトークン化
        self.encoded_texts = []
        for entry in data:

            ###################################################################
            # NEW: `format_input_phi`を使用し、レスポンステキストテンプレートを調整
            instruction_plus_input = format_input(entry)
            response_text = f"\n<|assistant|>:\n{entry['output']}"
            ###################################################################
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)


tokenizer = tiktoken.get_encoding("gpt2")
```

最後に、テストセットのレスポンスを収集する際の生成されたレスポンスの抽出方法も更新する必要があります：

```python
for i, entry in tqdm(enumerate(test_data), total=len(test_data)):

    input_text = format_input(entry)
    tokenizer=tokenizer

    token_ids = generate(
        model=model,
        idx=text_to_token_ids(input_text, tokenizer).to(device),
        max_new_tokens=256,
        context_size=BASE_CONFIG["context_length"],
        eos_id=50256
    )
    generated_text = token_ids_to_text(token_ids, tokenizer)

    # New: ###Response -> <|assistant|>に調整
    response_text = generated_text[len(input_text):].replace("<|assistant|>:", "").strip()

    test_data[i]["model_response"] = response_text
```

ご便宜のため、演習解答は[exercise_experiments.py](exercise_experiments.py)スクリプトに実装されており、次のように実行できます：

```bash
python exercise_experiments.py --exercise_solution phi3_prompt
```

出力：

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 3.71630220413208
   Validation loss: 3.6440994262695314
Ep 1 (Step 000000): Train loss 2.633, Val loss 2.622
...
Ep 2 (Step 000230): Train loss 0.424, Val loss 0.928
<|user|> Convert the active sentence to passive: 'The chef cooks the meal every day.' <|assistant|>: The meal is prepared every day by the chef....
Training completed in 1.50 minutes.
Plot saved as loss-plot-phi3-prompt.pdf
--------------------------------------------------
Generating responses
100% 110/110 [00:11<00:00,  9.27it/s]
Responses saved as instruction-data-with-response-phi3-prompt.json
Model saved as gpt2-medium355M-sft-phi3-prompt.pth
```

比較のため、元の第7章のファインチューニングコードは`python exercise_experiments.py --exercise_solution baseline`で実行できます。

Nvidia L4 GPUでは、上記のPhi-3プロンプトテンプレートを使用したコードの実行に1.5分かかることに注意してください。比較として、Alpacaスタイルテンプレートは1.80分かかります。そのため、Phi-3テンプレートはより短いモデル入力となるため、約17%高速です。

レスポンスが正しくフォーマットされているか確認するために、いくつかのレスポンスを見てみましょう：

```json
    {
        "instruction": "Rewrite the sentence using a simile.",
        "input": "The car is very fast.",
        "output": "The car is as fast as lightning.",
        "model_response": "The car is as fast as a cheetah."
    },
    {
        "instruction": "What type of cloud is typically associated with thunderstorms?",
        "input": "",
        "output": "The type of cloud typically associated with thunderstorms is cumulonimbus.",
        "model_response": "The type of cloud associated with thunderstorms is a cumulus cloud."
    },
    {
        "instruction": "Name the author of 'Pride and Prejudice'.",
        "input": "",
        "output": "Jane Austen.",
        "model_response": "The author of 'Pride and Prejudice' is Jane Austen."
    },
```

ご便宜のため、`python exercise_experiments.py`スクリプトにも実装されているOllama Llama 3手法を使用してパフォーマンスを評価できます。次のように実行できます：

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-phi3-prompt.json
```

出力：

```
Ollama running: True
Scoring entries: 100%|████████████████████████| 110/110 [01:08<00:00,  1.60it/s]
Number of scores: 110 of 110
Average score: 48.87
```

スコアは50に近く、これは以前にAlpacaスタイルプロンプトで達成したスコアと同程度です。

Phiプロンプトスタイルが優れている固有の利点や根拠はありませんが、以下の*ヒント*セクションで述べる注意点を除けば、より簡潔で効率的である可能性があります。

#### ヒント：特殊トークンの検討

- Phi-3プロンプトテンプレートには`<|user|>`や`<|assistant|>`などの特殊トークンが含まれており、これらはGPT-2トークナイザーにとって準最適である可能性があることに注意してください
- GPT-2トークナイザーは`<|endoftext|>`を特殊トークン（トークンID 50256にエンコード）として認識しますが、前述のような他の特殊トークンの処理は非効率です
- 例えば、`<|user|>`は5つの個別のトークンID（27, 91, 7220, 91, 29）にエンコードされ、これは非常に非効率です
- `allowed_special`引数を介して`tiktoken`に新しい特殊トークンとして`<|user|>`を追加することもできますが、GPT-2の語彙は追加の修正なしにはそれを処理できないことに注意してください
- トークナイザーとLLMを特殊トークンを処理するように拡張する方法に興味がある場合は、[extend-tiktoken.ipynb](../../ch05/09_extending-tokenizers/extend-tiktoken.ipynb)のボーナス資料をご覧ください（これはここでは必須ではありませんが、好奇心旺盛な読者にとって興味深い/ボーナス的な検討事項です）
- さらに、プロンプトテンプレートのこれらの特殊トークンを語彙を介してサポートするモデルは、より効率的で全体的に優れたパフォーマンスを発揮する可能性があると仮説を立てることができます

&nbsp;
## 演習 7.2：命令と入力のマスク

次の図に示すように命令をマスクアウトするには、`InstructionDataset`クラスと`custom_collate_fn`に軽微な変更を加える必要があります。

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch07_compressed/mask-instructions.webp" width=600px>

In [4]:
# This `format_input` function is copied from the original chapter 7 code

def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""

    return instruction_text + input_text

`InstructionDataset`クラスを変更して命令の長さを収集し、これをcollate関数で使用してターゲットで命令コンテンツの位置を特定し、collate関数をコーディングする際に使用できます：

In [5]:
import torch
from torch.utils.data import Dataset


class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data

        ##########################################################################################
        # New: Separate list for instruction lengths
        self.instruction_lengths = []
        ##########################################################################################
        
        self.encoded_texts = []
        
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            
            self.encoded_texts.append(
                tokenizer.encode(full_text)
            )

            ##########################################################################################
            # New: collect instruction lengths
            instruction_length = len(tokenizer.encode(instruction_plus_input))
            self.instruction_lengths.append(instruction_length)
            ##########################################################################################
            
    def __getitem__(self, index):
        # New: return both instruction lengths and texts separately
        return self.instruction_lengths[index], self.encoded_texts[index]

    def __len__(self):
        return len(self.data)

In [6]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

次に、`InstructionDataset`データセットの変更により、各`batch`が単なる`item`ではなく`(instruction_length, item)`を含むタプルとなる`custom_collate_fn`を更新します。さらに、ターゲットIDリストで対応する命令トークンをマスクします。

In [7]:
def custom_collate_fn(
    batch,
    pad_token_id=50256,
    ignore_index=-100,
    allowed_max_length=None,
    device="cpu"
):
    # Find the longest sequence in the batch
    batch_max_length = max(len(item)+1 for instruction_length, item in batch)   # New: batch is now a tuple

    # Pad and prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for instruction_length, item in batch:  # New: batch is now a tuple
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]
        # Pad sequences to max_length
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))
        inputs = torch.tensor(padded[:-1])  # Truncate the last token for inputs
        targets = torch.tensor(padded[1:])  # Shift +1 to the right for targets

        # Replace all but the first padding tokens in targets by ignore_index
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:
            targets[indices[1:]] = ignore_index

        ##########################################################################################
        # New: Mask all input and instruction tokens in the targets
        targets[:instruction_length-1] = -100
        ##########################################################################################
        
        # Optionally truncate to maximum sequence length
        if allowed_max_length is not None:
            inputs = inputs[:allowed_max_length]
            targets = targets[:allowed_max_length]
        
        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Convert list of inputs and targets to tensors and transfer to target device
    inputs_tensor = torch.stack(inputs_lst).to(device)
    targets_tensor = torch.stack(targets_lst).to(device)

    return inputs_tensor, targets_tensor

以下のサンプルデータで試してみましょう：

In [8]:
sample_data = [
    {'instruction': "What is an antonym of 'complicated'?", 'input': '', 'output': "An antonym of 'complicated' is 'simple'."},
    {'instruction': 'Sort the following list in alphabetical order.', 'input': 'Zebra, Elephant, Crocodile', 'output': 'Crocodile, Elephant, Zebra'},
    {'instruction': 'Arrange the given numbers in descending order.', 'input': '5, 12, 8, 3, 15', 'output': '15, 12, 8, 5, 3.'}
]

In [9]:
from torch.utils.data import DataLoader

train_dataset = InstructionDataset(sample_data, tokenizer)
train_loader = DataLoader(
    train_dataset,
    batch_size=len(sample_data),
    collate_fn=custom_collate_fn,
    num_workers=0
)

In [10]:
print("Train loader:")
for inputs, targets in train_loader:
    print(inputs.shape, targets.shape)

Train loader:
torch.Size([3, 64]) torch.Size([3, 64])


In [11]:
print("Inputs:\n", inputs[1])
print("\n\nTargets:\n", targets[1])

Inputs:
 tensor([21106,   318,   281, 12064,   326,  8477,   257,  4876,    13, 19430,
          257,  2882,   326, 20431, 32543,   262,  2581,    13,   198,   198,
        21017, 46486,    25,   198, 42758,   262,  1708,  1351,   287, 24830,
          605,  1502,    13,   198,   198, 21017, 23412,    25,   198,    57,
        37052,    11, 42651,    11,  9325, 19815,   576,   198,   198, 21017,
        18261,    25,   198,    34, 12204,   375,   576,    11, 42651,    11,
         1168, 37052, 50256, 50256])


Targets:
 tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,   198,   198, 21017, 18261,
           25,   198,    34, 12204,   375,   576,    11, 42651,    11,  1168,
      

`targets`テンソルに基づいて、命令とパディングトークンの両方が-100プレースホルダートークンを使用してマスクされていることがわかります。
入力が正しく見えるか確認するために、入力をデコードしてみましょう：

In [12]:
print(tokenizer.decode(list(inputs[1])))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Sort the following list in alphabetical order.

### Input:
Zebra, Elephant, Crocodile

### Response:
Crocodile, Elephant, Zebra<|endoftext|><|endoftext|>


次に、マスクされていないターゲットトークンIDをデコードしてみましょう：

In [13]:
non_masked_targets = targets[1][targets[1] != -100]

print(tokenizer.decode(list(non_masked_targets)))



### Response:
Crocodile, Elephant, Zebra<|endoftext|>


上記に示すように、マスクされていないターゲットトークンは意図した通り`"Instruction"`と`"Input"`フィールドを除外しています。これで、このマスキング戦略を使用してファインチューニングされたときのLLMのパフォーマンスを確認するために、修正されたコードを実行できます。

ご便宜のため、`exercise_experiments.py`コードを使用して次のような比較を実行できます：

```bash
python exercise_experiments.py --exercise_solution mask_instructions
```

出力：

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 2.280539035797119
   Validation loss: 2.262560224533081
Ep 1 (Step 000000): Train loss 1.636, Val loss 1.620
...
Ep 2 (Step 000230): Train loss 0.143, Val loss 0.727
...
Training completed in 1.77 minutes.
Plot saved as loss-plot-mask-instructions.pdf
--------------------------------------------------
Generating responses
100% 110/110 [02:10<00:00,  1.19s/it]
Responses saved as instruction-data-with-response-mask-instructions.json
Model saved as gpt2-medium355M-sft-mask-instructions.pth
```

次に、結果のLLMのパフォーマンスを評価しましょう：

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-mask-instructions.json
```

```
Ollama running: True
Scoring entries: 100%|██████████████████████████████████████████████████████████████████████████████████████| 110/110 [01:23<00:00,  1.31it/s]
Number of scores: 110 of 110
Average score: 47.73
```

スコアに基づいて、命令マスキングの性能はわずかに劣っており、これは「Instruction Tuning With Loss Over Instructions」論文（https://arxiv.org/abs/2405.14394）での観察と一致しています

&nbsp;
## 演習 7.3：元のAlpacaデータセットでのファインチューニング

元のStanford Alpacaデータセット（[https://github.com/tatsu-lab/stanford_alpaca](https://github.com/tatsu-lab/stanford_alpaca)）でモデルをファインチューニングするには、ファイルURLを

```python
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch07/01_main-chapter-code/instruction-data.json"
```

から

```python
url = "https://raw.githubusercontent.com/tatsu-lab/stanford_alpaca/main/alpaca_data.json"
```

に変更するだけです。

データセットには52kエントリ（第7章で使用したものの50倍）が含まれており、エントリは第7章で使用したものより長いことに注意してください。
そのため、GPUでトレーニングを実行することを強く推奨します。

メモリ不足エラーが発生した場合は、バッチサイズを8から4、2、または1に減らすことを検討してください。バッチサイズを下げることに加えて、`allowed_max_length`を1024から512または256に下げることも検討できます。

ご便宜のため、`exercise_experiments.py`コードを使用して、バッチサイズ4と`allowed_max_length`を512に設定して52k Alpacaデータセットでモデルをファインチューニングできます：

```bash
python exercise_experiments.py --exercise_solution alpaca_52k
```

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 44201
Validation set length: 2601
Test set length: 5200
--------------------------------------------------
Device: cuda
--------------------------------------------------
...
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Initial losses
   Training loss: 3.3681655883789063
   Validation loss: 3.4122894287109373
Ep 1 (Step 000000): Train loss 2.477, Val loss 2.750
...
Ep 2 (Step 022095): Train loss 0.761, Val loss 1.557
...
Training completed in 196.38 minutes.
Plot saved as loss-plot-alpaca52k.pdf
--------------------------------------------------
Generating responses
100% 5200/5200 [2:56:33<00:00,  2.04s/it]
Responses saved as instruction-data-with-response-alpaca52k.json
Model saved as gpt2-medium355M-sft-alpaca52k.pth
```

以下は、生成されたモデルレスポンスを含むAlpacaデータセットからのいくつかの例です：

```json
[
    {
        "instruction": "Edit the following sentence to increase readability: \"He made a huge effort and was so successful.\"",
        "input": "",
        "output": "He exerted a tremendous effort, and thus enjoyed great success.",
        "model_response": "He put in an immense effort and was rewarded with success."
    },
    {
        "instruction": "Rewrite the following sentence to make it more concise: \"I was displeased with the result of the experiment that I conducted.\"",
        "input": "",
        "output": "I was unhappy with my experiment's outcome.",
        "model_response": "I was displeased with the results of the experiment."
    },
    {
        "instruction": "How can we build a more efficient GPT model?",
        "input": "",
        "output": "We can build a more efficient GPT model by optimizing the architecture of the model, using smaller model sizes and training with fewer parameters. We can also leverage techniques such as knowledge distillation, transfer learning, dynamic sparsity and hybrid computing to further improve the efficiency of the model.",
        "model_response": "Building a more efficient GPT model requires careful planning and optimization. First, it is important to identify the target language and the context in which the model is used. Then, it is important to select the appropriate model architecture, such as backpropagation, hyperparameters, and hyperparameters. Finally, it is important to select the appropriate model weights and optimizers, such as backpropagation, hyperparameters, and hyperparameters."
    },
]

最後に、[ollama_evaluate.py](ollama_evaluate.py)ユーティリティ関数を使用してファインチューニングされたLLMを評価できます：

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-alpaca52k.json
```

```
Scoring entries: 100%|████████████████████| 5200/5200 [1:07:52<00:00, 1.28it/s]
Number of scores: 5188 of 5200
Average score: 48.16
```

スコアは、この章で使用したデータセットで得られたスコアよりもわずかに低くなっています。ただし、Alpacaテストセットには、メイン章で使用したデータセットよりも多様で、一部はより困難な命令が含まれていることに注意してください。

## 演習 7.4：LoRAによるパラメータ効率的ファインチューニング

LoRAを使用してモデルを命令ファインチューニングするには、付録Eから関連するクラスと関数を使用します：

```python
from appendix_E import LoRALayer, LinearWithLoRA, replace_linear_with_lora
```

次に、セクション7.5のモデル読み込みコードの下に以下のコード行を追加します：


```python
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters before: {total_params:,}")

for param in model.parameters():
    param.requires_grad = False

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters after: {total_params:,}")
replace_linear_with_lora(model, rank=16, alpha=16)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable LoRA parameters: {total_params:,}")
model.to(device)
```

ご便宜のため、`exercise_experiments.py`コードを使用して、rank 16とalpha 16のLoRAでモデルをファインチューニングできます：

```bash
python exercise_experiments.py --exercise_solution lora
```

出力：

```
matplotlib version: 3.7.1
tiktoken version: 0.7.0
torch version: 2.3.0+cu121
tqdm version: 4.66.4
tensorflow version: 2.15.0
--------------------------------------------------
Training set length: 935
Validation set length: 55
Test set length: 110
--------------------------------------------------
Device: cuda
--------------------------------------------------
File already exists and is up-to-date: gpt2/355M/checkpoint
File already exists and is up-to-date: gpt2/355M/encoder.json
File already exists and is up-to-date: gpt2/355M/hparams.json
File already exists and is up-to-date: gpt2/355M/model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2/355M/model.ckpt.index
File already exists and is up-to-date: gpt2/355M/model.ckpt.meta
File already exists and is up-to-date: gpt2/355M/vocab.bpe
Loaded model: gpt2-medium (355M)
--------------------------------------------------
Total trainable parameters before: 406,286,336
Total trainable parameters after: 0
Total trainable LoRA parameters: 7,898,384
Initial losses
   Training loss: 3.7684114456176756
   Validation loss: 3.7619335651397705
Ep 1 (Step 000000): Train loss 2.509, Val loss 2.519
...
Ep 2 (Step 000230): Train loss 0.308, Val loss 0.652
...
--------------------------------------------------
Generating responses
100% 110/110 [01:52<00:00,  1.03s/it]
Responses saved as instruction-data-with-response-lora.json
Model saved as gpt2-medium355M-sft-lora.pth
```

比較のため、元の第7章のファインチューニングコードは`python exercise_experiments.py --exercise_solution baseline`で実行できます。

Nvidia L4 GPUでは、上記のLoRAを使用したコードの実行に1.30分かかることに注意してください。比較として、ベースラインは1.80分かかります。そのため、LoRAは約28%高速です。


ご便宜のため、`python exercise_experiments.py`スクリプトにも実装されているOllama Llama 3手法を使用してパフォーマンスを評価できます。次のように実行できます：

```bash
python ollama_evaluate.py --file_path instruction-data-with-response-lora.json
```

出力：

```
Ollama running: True
Scoring entries: 100%|████████████████████████| 110/110 [01:13<00:00,  1.50it/s]
Number of scores: 110 of 110
Average score: 50.23
```

スコアは50付近で、元のモデルと同程度です。